In [13]:
import pandas as pd

#Cleaning players_data
   
players_data = pd.read_csv("Dataset/players.csv")
#players_data.dropna(thresh=2, inplace=True)
players_data = players_data[players_data.pos.notnull()]
players_data.rename(columns={"weight": "w"}, inplace=True)
players_data.rename(columns={"height": "weight"}, inplace=True)
players_data.rename(columns={"w": "height"}, inplace=True)
players_data.drop(players_data[players_data['weight'] < 40].index, inplace = True)
players_data.drop(players_data[players_data['height'] < 140].index, inplace = True)
players_data.drop(columns=["firstseason","lastseason","deathDate"],inplace=True)
players_data.reset_index()
players_data.describe()

,weight,height
count,682.000000,682.000000
mean,72.665689,172.476540
std,3.137050,21.279833
min,63.000000,140.000000
25%,70.000000,157.250000
50%,73.000000,170.000000
75%,75.000000,185.000000
max,80.000000,254.000000


In [14]:
teams_data = pd.read_csv("Dataset/teams.csv")


In [15]:
players_teams_data = pd.read_csv("Dataset/players_teams.csv")
awards_players_data = pd.read_csv("Dataset/awards_players.csv")
players_teams_data['awards'] = 0
for idx, player in players_teams_data.iterrows():
    awards_until_target_season = awards_players_data[(awards_players_data['playerID'] == player['playerID']) & (awards_players_data['year'] < player['year'])]
    if(not awards_until_target_season.empty):
        players_teams_data.loc[[idx], 'awards'] = len(awards_until_target_season)
print(players_teams_data[players_teams_data["awards"]>7] )
for idx, team in teams_data.iterrows():
        team_awards_until_target_season = players_teams_data[(players_teams_data["year"] == team["year"] )&( players_teams_data["tmID"] == team["tmID"])]
        team_awards_num = team_awards_until_target_season["awards"].sum()
        teams_data.loc[[idx], 'awards'] = team_awards_num
teams_data[teams_data["tmID"]=="LAS"]        

        playerID  year  stint tmID  GP  GS  minutes  points  oRebounds  \
910   leslili01w     9      0  LAS  33  33     1059     497         84   
911   leslili01w    10      0  LAS  23  21      638     354         56   
1582  swoopsh01w     8      0  HOU   3   3      106      23          3   
1583  swoopsh01w     9      0  SEA  29  25      706     205         26   

      dRebounds  ...  PostTurnovers  PostPF  PostfgAttempted  PostfgMade  \
910         209  ...             26      29               64          33   
911          95  ...             13      26               84          38   
1582         14  ...              0       0                0           0   
1583        100  ...              5       7               25           8   

      PostftAttempted  PostftMade  PostthreeAttempted  PostthreeMade  PostDQ  \
910                24          15                   4              2       2   
911                39          24                   0              0       1   
1582    

,year,tmID,confID,rank,playoff,firstRound,semis,finals,name,o_fgm,...,homeW,homeL,awayW,awayL,confW,confL,min,attend,arena,awards
42,1,LAS,WE,1,Y,W,L,NaN,Los Angeles Sparks,861,...,15,1,13,3,17,4,6450,105005,Staples Center,0.0
43,2,LAS,WE,1,Y,W,W,W,Los Angeles Sparks,916,...,16,0,12,4,19,2,6475,148446,Staples Center,0.0
44,3,LAS,WE,1,Y,W,W,W,Los Angeles Sparks,891,...,12,4,13,3,17,4,6475,186410,Staples Center,3.0
45,4,LAS,WE,1,Y,W,W,L,Los Angeles Sparks,894,...,11,6,13,4,17,7,6900,157934,Staples Center,6.0
46,5,LAS,WE,1,Y,L,NaN,NaN,Los Angeles Sparks,904,...,15,2,10,7,16,6,6975,177269,Staples Center,6.0
47,6,LAS,WE,4,Y,L,NaN,NaN,Los Angeles Sparks,873,...,11,6,6,11,12,10,6800,150521,Staples Center,8.0
48,7,LAS,WE,1,Y,W,L,NaN,Los Angeles Sparks,962,...,15,2,10,7,15,5,6825,141282,Staples Center,8.0
49,8,LAS,WE,7,N,NaN,NaN,NaN,Los Angeles Sparks,940,...,5,12,5,12,6,16,6900,147810,Staples Center,3.0
120,9,LAS,WE,3,Y,W,L,NaN,Los Angeles Sparks,954,...,12,5,8,9,12,8,7000,161639,Staples Center,10.0
133,10,LAS,WE,3,Y,W,L,NaN,Los Angeles Sparks,973,...,11,6,7,10,11,9,6925,176587,Staples Center,16.0
